In [1]:
import torch

# OpenNMT-pytorch

#### ref
* SKplanet Tacademy
    * https://github.com/Parkchanjun/SK_MT_FOR_EVERYONE 
* https://opennmt.net/OpenNMT-py/ : documents
* https://github.com/OpenNMT/OpenNMT-py
    * onmt에 핵심 소스, tools에 필요한 도구들이 존재(tokenizer, bleu측정 등)
* https://forum.opennmt.net/ : 모르는 것들 질문하면 개발자들이 직접 답해주는 커뮤니티 
* 버전 문제가 있기 때문에 OpenNMT-py github에서 파일을 확인해가면서 사용해야함

In [1]:
!git clone https://github.com/OpenNMT/OpenNMT-py

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 17374, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 17374 (delta 182), reused 290 (delta 162), pack-reused 17044
Receiving objects: 100% (17374/17374), 273.66 MiB | 9.11 MiB/s, done.
Resolving deltas: 100% (12482/12482), done.
Checking out files: 100% (231/231), done.


In [5]:
# setup install
!python OpenNMT-py/setup.py install --user

running install
running bdist_egg
running egg_info
writing OpenNMT_py.egg-info/PKG-INFO
writing dependency_links to OpenNMT_py.egg-info/dependency_links.txt
writing entry points to OpenNMT_py.egg-info/entry_points.txt
writing requirements to OpenNMT_py.egg-info/requires.txt
writing top-level names to OpenNMT_py.egg-info/top_level.txt
reading manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
writing manifest file 'OpenNMT_py.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/entry_points.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying OpenNMT_py.egg-info/requires.txt -> build/bdist

Best match: tensorboard-data-server 0.6.1
Processing tensorboard_data_server-0.6.1-py3-none-any.whl
Installing tensorboard_data_server-0.6.1-py3-none-any.whl to /home/work/.local/lib/python3.6/site-packages
Adding tensorboard-data-server 0.6.1 to easy-install.pth file

Installed /home/work/.local/lib/python3.6/site-packages/tensorboard_data_server-0.6.1-py3.6.egg
Searching for google-auth-oauthlib<0.5,>=0.4.1
Reading https://pypi.org/simple/google-auth-oauthlib/
Best match: google-auth-oauthlib 0.4.4
Processing google_auth_oauthlib-0.4.4-py2.py3-none-any.whl
Installing google_auth_oauthlib-0.4.4-py2.py3-none-any.whl to /home/work/.local/lib/python3.6/site-packages
Adding google-auth-oauthlib 0.4.4 to easy-install.pth file
Installing google-oauthlib-tool script to /home/work/.local/bin

Installed /home/work/.local/lib/python3.6/site-packages/google_auth_oauthlib-0.4.4-py3.6.egg
Searching for Flask==1.1.2
Best match: Flask 1.1.2
Adding Flask 1.1.2 to easy-install.pth file
Installing flas

In [9]:
# 환경 설정이 중요. requirements.txt에 환경이 정해져 있음
!pip install -r OpenNMT-py/requirements.opt.txt

Defaulting to user installation because normal site-packages is not writeable
  Cloning git://github.com/NVIDIA/apex.git (to revision 700d6825e205732c1d6be511306ca4e595297070) to /tmp/pip-req-build-y2a6b0g2
  Running command git clone -q git://github.com/NVIDIA/apex.git /tmp/pip-req-build-y2a6b0g2
  Running command git checkout -q 700d6825e205732c1d6be511306ca4e595297070
  Running command git submodule update --init --recursive -q
  Created wheel for apex: filename=apex-0.1-py3-none-any.whl size=192739 sha256=de4c3a9732054c0c6cc549cee4c6d9da1b6d5202e2e9f40efc14a09c93daeaa6
  Stored in directory: /home/work/.cache/pip/wheels/8d/f8/65/1a008861e573ad6c957b50e46f7fc63ce4cbce157fad9d8a38
Successfully built apex


### process
1. data 수집 : 제공하는 toy data 사용
2. 정제, 병렬 코퍼스 필터링(생략) : 제공하는 toy data 사용
3. 서브워드 분리
4. 학습
5. 번역
6. Detokenization 
7. 평가

### Subword Tokenization (BPE) 
* i => input
* o => output
* s => Symbol
* learn_bpe => bpe모델(code)를 만듬
* apply_bpe => bpe모델을 적용
* src-train, src-val,test ==> src.code에서 적용 
* tgt-train,tgt-val ==> tgt.code에서 적용

In [10]:
# src train을 가지고 bpe code(src.code)를 생성
# S : 일반적으로 32000을 준다고 함.
!python OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src.code -s 10000

In [11]:
# trg train을 가지고 bpe code(trg.code)를 생성
# S : 일반적으로 32000을 준다고 함.
!python OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt.code -s 10000

In [12]:
# 생성한 bpe code(model)을 적용
# c : 이전에 learn_bpe로 생성한 code
# i : token화 할 input -> data src-train.txt
# o : token화 한 output -> src-train-bpe.txt
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src-train-bpe.txt

In [13]:
# src.code는 동일, val data 적용
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-val.txt -o OpenNMT-py/data/src-val-bpe.txt

In [14]:
# src.code는 동일, test data 적용
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-test.txt -o OpenNMT-py/data/src-test-bpe.txt

In [15]:
# tgt.code는 동일, train data 적용
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt-train-bpe.txt

In [16]:
# tgt.code는 동일, val data 적용
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-val.txt -o OpenNMT-py/data/tgt-val-bpe.txt

In [2]:
open('OpenNMT-py/data/tgt-val.txt',encoding='utf-8').read()

'Keine befreiende Novelle für Tymoshenko durch das Parlament\nDas ukrainische Parlament verweigerte heute den Antrag , im Rahmen einer Novelle des Strafgesetzbuches denjenigen Paragrafen abzuschaffen , auf dessen Grundlage die Oppositionsführerin Yulia Timoshenko verurteilt worden war .\nDie Neuregelung , die den Weg zur Befreiung der inhaftierten Expremierministerin hätte ebnen können , lehnten die Abgeordneten bei der zweiten Lesung des Antrags auf Milderung der Strafen für wirtschaftliche Delikte ab .\nTimoshenko war im Oktober wegen des Abschlusses eines angeblich nachteiligen Vertrags mit Russland über den Einkauf von Erdgas zu sieben Jahren Haft verurteilt worden .\nDas Urteil ist noch nicht rechtskräftig , im Dezember soll die Berufung der Verurteilten vor Gericht verhandelt werden .\nTimoshenko selbst bezeichnet das Urteil als politische Rache des Regimes und auch im Westen ließ der Prozess den Verdacht der Voreingenommenheit des Gerichts aufkommen .\nDer Antrag , Paragraf 365 

In [3]:
## @@로 나뉘어져 있음 -> 후에 detokenization으로 제거해야함 
open('OpenNMT-py/data/tgt-val-bpe.txt',encoding='utf-8').read()

'K@@ eine be@@ frei@@ ende Nov@@ elle für Ty@@ mo@@ sh@@ en@@ ko durch das Parlament\nDas u@@ kra@@ in@@ ische Parlament ver@@ wei@@ ger@@ te heute den Antrag , im Rahmen einer Nov@@ elle des Straf@@ gesetz@@ bu@@ ches den@@ jenigen Para@@ gra@@ fen abzu@@ schaffen , auf dessen Grundlage die Opposi@@ tions@@ führ@@ erin Y@@ u@@ li@@ a Ti@@ mo@@ sh@@ en@@ ko verurteilt worden war .\nDie Neur@@ e@@ gelung , die den Weg zur Be@@ frei@@ ung der in@@ haf@@ tierten Ex@@ pre@@ mier@@ min@@ ister@@ in hätte eb@@ nen können , leh@@ nten die Abgeordneten bei der zweiten Lesung des An@@ tra@@ gs auf Mil@@ derung der Stra@@ fen für wirtschaftliche De@@ li@@ kte ab .\nTi@@ mo@@ sh@@ en@@ ko war im Oktober wegen des Ab@@ sch@@ luss@@ es eines angeb@@ lich nach@@ teili@@ gen Vertrags mit Russland über den Eink@@ auf von Erd@@ g@@ as zu sieben Jahren Haf@@ t verurteilt worden .\nDas Ur@@ teil ist noch nicht rechts@@ kräf@@ tig , im Dezember soll die Beru@@ fung der Ver@@ ur@@ teil@@ ten vor Ger@@ icht

### Preprocess the data

* 학습에 용이하도록 .txt 파일 형식을 바꿔줘야함
* preprocess.py를 사용해서 bpe.txt 파일과 매핑. 
    * train_src
    * train_tgt
    * valid_src
    * valid_tgt
* demo 라는 이름으로 저장
    * demo.pt 형식으로 파일 생성
* vocab size : 32000

In [6]:
# vocab 생성
!python OpenNMT-py/build_vocab.py -config OpenNMT-py/toy_data.yaml -n_sample 10000 --user

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-06-09 16:47:18,091 INFO] Counter vocab from 10000 samples.
[2021-06-09 16:47:18,091 INFO] Build vocab on 10000 transformed examples/corpus.
[2021-06-09 16:47:18,101 INFO] corpus_1's transforms: TransformPipe()
[2021-06-09 16:47:18,349 WARNING] Empty line exists in corpus_1#4.
[2021-06-09 16:47:18,804 INFO] Counters src:9911
[2021-06-09 16:47:18,804 INFO] Counters tgt:10152
Traceback (most recent call last):
  File "OpenNMT-py/build_vocab.py", line 6, in <module>
    main()
  File "/home/work/dataset/WAT/OpenNMT-py/onmt/bin/build_vocab.py", line 66, in main
    build_vocab_main(opts)
  File "/home/work/dataset/WAT/OpenNMT-py/onmt/bin/build_vocab.py", line 53, in build_vocab_main
    save_counter(src_counter, opts.src_vocab)
  File "/home/work/dataset/WAT/OpenNMT-py/onmt/bin/build_vocab.py", line 42, in save_counter
    check_path(save_path, exist_ok=opts.overwrite, log=logger.warning)
  File "/home/work/dataset/

In [7]:
# 기본 모델 RNN enc-dec로 train
!python OpenNMT-py/train.py -config OpenNMT-py/toy_data.yaml

[2021-06-09 16:47:39,172 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-06-09 16:47:39,174 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-06-09 16:47:39,174 INFO] Missing transforms field for valid data, set to default: [].
[2021-06-09 16:47:39,175 INFO] Parsed 2 corpora from -data.
[2021-06-09 16:47:39,176 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-06-09 16:47:39,176 INFO] Loading vocab from text file...
[2021-06-09 16:47:39,176 INFO] Loading src vocabulary from OpenNMT-py/data/run/example.vocab.src
[2021-06-09 16:47:39,237 INFO] Loaded src vocab has 9911 tokens.
[2021-06-09 16:47:39,245 INFO] Loading tgt vocabulary from OpenNMT-py/data/run/example.vocab.tgt
[2021-06-09 16:47:39,306 INFO] Loaded tgt vocab has 10152 tokens.
[2021-06-09 16:47:39,315 INFO] Building fields with vocab in counters...
[2021-06-09 16:47:39,333 INFO]  * tgt vocab size: 10156.
[2021-06-09 16:47:39,350 INFO]  * 

In [8]:
# 번역문 생성하기 
!python OpenNMT-py/translate.py -model OpenNMT-py/data/run/model_step_1000.pt -src OpenNMT-py/data/src-test.txt -output OpenNMT-py/data/pred_1000.txt -gpu 0 -verbose

[2021-06-09 17:00:20,981 INFO] Translating shard 0.
[2021-06-09 17:00:21,320 INFO] 
SENT 1: ['Orlando', 'Bloom', 'and', 'Miranda', 'Kerr', 'still', 'love', 'each', 'other']
PRED 1: In der Nähe der Kommission in der Nähe der Kommission .
PRED SCORE: -23.8011

[2021-06-09 17:00:21,321 INFO] 
SENT 2: ['Actors', 'Orlando', 'Bloom', 'and', 'Model', 'Miranda', 'Kerr', 'want', 'to', 'go', 'their', 'separate', 'ways', '.']
PRED 2: In der Nähe der Kommission in der Nähe der Kommission in der Nähe der Kommission .
PRED SCORE: -32.4905

[2021-06-09 17:00:21,321 INFO] 
SENT 3: ['However', ',', 'in', 'an', 'interview', ',', 'Bloom', 'has', 'said', 'that', 'he', 'and', 'Kerr', 'still', 'love', 'each', 'other', '.']
PRED 3: Es gibt es wichtig , dass die Europäische Union in der Europäischen Union in der Europäischen Union .
PRED SCORE: -28.5482

[2021-06-09 17:00:21,321 INFO] 
SENT 4: ['Miranda', 'Kerr', 'and', 'Orlando', 'Bloom', 'are', 'parents', 'to', 'two-year-old', 'Flynn', '.']
PRED 4: In der N

[2021-06-09 17:00:21,514 INFO] 
SENT 31: ['They', 'know', 'their', 'business', 'better', 'than', 'anyone', 'else', '.']
PRED 31: Wenn Sie uns in der Nähe .
PRED SCORE: -14.9366

[2021-06-09 17:00:21,514 INFO] 
SENT 32: ['For', 'flyers', 'it', 'is', 'about', 'more', 'elbow', 'room', ',', 'but', 'for', 'suppliers', 'it', 'is', 'increasingly', 'an', 'issue', 'that', 'could', 'affect', 'earnings', '.']
PRED 32: In der Nähe der Europäischen Union in der Nähe der Europäischen Union in der Nähe der Europäischen Union in der Europäischen Union .
PRED SCORE: -40.7320

[2021-06-09 17:00:21,514 INFO] 
SENT 33: ['Behind', 'the', 'dispute', 'is', 'a', 'race', 'for', 'plane', 'orders', 'with', 'at', 'least', '$', '700-billion', 'of', 'estimated', 'business', 'at', 'list', 'prices', 'in', 'coming', 'decades', ',', 'enough', 'to', 'tip', 'the', 'scales', 'of', 'U.S.', 'and', 'European', 'exports', '.']
PRED 33: In der Nähe der Kommission in der Nähe der Kommission in der Nähe der Europäischen Union in

[2021-06-09 17:00:21,671 INFO] 
SENT 61: ['However', ',', 'they', 'did', 'mention', 'that', 'there', 'were', 'various', 'witness', 'statements', 'that', 'first', 'had', 'to', 'be', 'assessed', '.']
PRED 61: Es gibt es wichtig , dass die Europäische Union in der Europäischen Union in der Europäischen Union .
PRED SCORE: -28.5094

[2021-06-09 17:00:21,671 INFO] 
SENT 62: ['What', 'is', 'certain', 'is', 'that', 'the', '19-year-old', 'female', 'driver', 'of', 'a', 'VW', 'Golf', 'was', 'driving', 'in', 'the', 'direction', 'of', 'Revensdorf', 'and', 'the', '38-year-old', 'man', 'from', 'Gettorf', 'came', 'towards', 'her', 'in', 'his', 'Hyundai', '.']
PRED 62: Wenn Sie die Kommission in der Europäischen Union in der Europäischen Union in der Europäischen Union in der Europäischen Union in der Europäischen Union in der Europäischen Union in der Europäischen Union .
PRED SCORE: -55.2981

[2021-06-09 17:00:21,671 INFO] 
SENT 63: ['The', 'two', 'vehicles', 'collided', ',', 'whereby', 'the', 'woma

[2021-06-09 17:00:21,884 INFO] 
SENT 91: ['The', 'firefighters', 'were', 'amazing', '.']
PRED 91: Das Europäische Parlament hat die Europäische Union .
PRED SCORE: -14.8661

[2021-06-09 17:00:21,884 INFO] 
SENT 92: ['It', 'was', 'really', 'daring', 'what', 'they', 'did', '.']
PRED 92: Es gibt jedoch in der Nähe .
PRED SCORE: -12.7234

[2021-06-09 17:00:21,885 INFO] 
SENT 93: ['We', 'are', 'just', 'so', 'grateful', 'and', 'every', 'single', 'one', 'of', 'them', 'was', 'absolutely', 'tremendous', '.']
PRED 93: Wir brauchen die Kommission in der Mitgliedstaaten in der Nähe der Kommission .
PRED SCORE: -24.8900

[2021-06-09 17:00:21,885 INFO] 
SENT 94: ['Mr', 'Alderson', ',', 'from', 'Keighley', ',', 'added:', '&quot;', 'We', 'were', 'scared', 'that', 'she', 'might', 'fall', 'but', 'she', 'stayed', 'there', '.', '&quot;']
PRED 94: Herr Präsident ! Herr Präsident !
PRED SCORE: -5.8772

[2021-06-09 17:00:21,885 INFO] 
SENT 95: ['The', 'firefighters', 'were', 'brilliant', '.']
PRED 95: Das Eu

[2021-06-09 17:00:22,115 INFO] 
SENT 121: ['&quot;', 'Not', 'only', 'have', 'you', 'kept', 'countless', 'records', 'for', 'us', ',', 'but', 'you', 'have', 'also', 'done', 'so', 'much', 'running', 'around', 'for', 'us', ',', 'and', 'for', 'this', 'we', 'offer', 'our', 'sincere', 'thanks', ',', '&quot;', 'said', 'Choir', 'Chairman', 'Erich', 'Schlotmann', '.']
PRED 121: Wenn Sie die Kommission in der Nähe der Kommission in der Nähe der Kommission in der Nähe der Kommission in der Nähe der Kommission in der Nähe der Kommission in der Nähe der Welt in der Welt .
PRED SCORE: -75.3728

[2021-06-09 17:00:22,115 INFO] 
SENT 122: ['The', 'awards', 'for', 'participation', 'in', 'the', 'performances', 'and', 'choir', 'rehearsals', 'by', 'means', 'of', 'the', 'presentation', 'of', '&quot;', 'activity', 'posies', '&quot;', ',', 'this', 'year', 'went', 'to', 'Choir', 'Director', 'Dieter', 'Schulze', ',', 'with', '44', 'attendance', 'marks', ',', 'followed', 'by', 'Honorary', 'Chairman', 'Horst', 'Pi

...

[2021-06-09 17:00:37,374 INFO] 
SENT 2611: ['However', ',', 'Washington', '&apos;s', 'growing', 'interest', 'in', 'the', 'Pacific', 'suggests', 'Britain', 'will', 'be', 'left', 'out', 'in', 'the', 'Atlantic', 'cold', '.']
PRED 2611: Es gibt es wichtig , dass die Europäische Union in der Europäischen Union in der Europäischen Union .
PRED SCORE: -28.8650

[2021-06-09 17:00:37,375 INFO] 
SENT 2612: ['Where', 'does', 'this', 'leave', 'Scotland', '?']
PRED 2612: Wenn Sie uns nicht .
PRED SCORE: -12.2555

[2021-06-09 17:00:37,375 INFO] 
SENT 2613: ['We', 'can', 'choose', 'to', 'be', 'a', 'region', 'of', '(essentially)', 'Little', 'England', '.']
PRED 2613: Wir brauchen die Kommission in der Nähe der Europäischen Union .
PRED SCORE: -18.6051

[2021-06-09 17:00:37,375 INFO] 
SENT 2614: ['Or', 'we', 'can', 'defend', 'our', 'own', 'economic', 'interests', '-', 'which', 'includes', 'telling', 'Berlin', 'and', 'Brussels', 'where', 'to', 'get', 'off', '.']
PRED 2614: In der Nähe der Europäischen U

[2021-06-09 17:00:37,518 INFO] 
SENT 2641: ['Johanna', 'Rahner', ',', 'born', 'in', '1962', 'in', 'Baden-Baden', ',', 'studied', 'Catholic', 'theology', 'and', 'biology', 'from', '1982', 'to', '1989', 'at', 'the', 'Albert', 'Ludwig', '&apos;s', 'University', 'in', 'Freiburg', '.']
PRED 2641: In der Nähe der Kommission in der Nähe der Europäischen Union in der Nähe der Kommission in der Nähe der Europäischen Union in der Europäischen Union .
PRED SCORE: -49.7030

[2021-06-09 17:00:37,519 INFO] 
SENT 2642: ['In', '1997', 'she', 'graduated', 'as', 'a', 'doctor', 'of', 'theology', ',', 'also', 'in', 'Freiburg', '.']
PRED 2642: In der Nähe der Kommission in der Europäischen Union in der Europäischen Union .
PRED SCORE: -22.4830

[2021-06-09 17:00:37,519 INFO] 
SENT 2643: ['A', 'post-doctoral', 'lecturing', 'qualification', 'in', 'fundamental', 'theology', 'and', 'ecumenical', 'theology', 'followed', 'in', '2003', 'at', 'the', 'Westfalian', 'Wilhelm', '&apos;s', 'University', '.']
PRED 2643:

[2021-06-09 17:00:37,710 INFO] 
SENT 2671: ['It', 'is', 'not', 'yet', 'clear', 'how', 'the', 'new', 'owner', 'will', 'use', 'the', 'former', 'Kindergarten', '.']
PRED 2671: Es gibt es wichtig , dass die Europäische Union in der Europäischen Union .
PRED SCORE: -21.0272

[2021-06-09 17:00:37,710 INFO] 
SENT 2672: ['The', 'Mayor', 'considers', 'it', 'the', 'right', 'decision', 'that', 'Winterlingen', 'has', 'handed', 'over', 'responsibility', 'for', 'the', 'building', 'to', 'someone', 'else', 'before', 'the', 'winter', ',', 'and', 'thus', 'will', 'no', 'longer', 'have', 'to', 'pay', 'for', 'the', 'upkeep', 'of', 'the', 'building', ',', 'such', 'as', 'heating', 'it', ',', 'tending', 'to', 'the', 'garden', 'and', 'clearing', 'the', 'pavement', '.']
PRED 2672: Das Europäische Parlament hat die Kommission in der Nähe der Kommission .
PRED SCORE: -23.9908

[2021-06-09 17:00:37,710 INFO] 
SENT 2673: ['&quot;', 'This', 'expense', 'has', 'now', 'been', 'offloaded', ',', '&quot;', 'Maier', 'was',

[2021-06-09 17:00:37,899 INFO] 
SENT 2701: ['They', '&apos;re', 'not', 'supposed', 'to', 'restart', 'them', 'until', 'the', 'planes', 'reach', '10,000', 'feet', 'and', 'the', 'captain', 'gives', 'the', 'go-ahead', '.']
PRED 2701: Wenn Sie die Kommission in der Nähe der Kommission .
PRED SCORE: -21.3193

[2021-06-09 17:00:37,900 INFO] 
SENT 2702: ['Passengers', 'are', 'supposed', 'to', 'turn', 'their', 'devices', 'off', 'again', 'as', 'the', 'plane', 'descends', 'to', 'land', 'and', 'not', 'restart', 'them', 'until', 'the', 'plane', 'is', 'on', 'the', 'ground', '.']
PRED 2702: In der Nähe der Kommission in der Nähe der Kommission in der Nähe der Kommission in der Nähe der Kommission .
PRED SCORE: -41.9978

[2021-06-09 17:00:37,900 INFO] 
SENT 2703: ['Under', 'the', 'new', 'guidelines', ',', 'airlines', 'whose', 'planes', 'are', 'properly', 'protected', 'from', 'electronic', 'interference', 'may', 'allow', 'passengers', 'to', 'use', 'the', 'devices', 'during', 'takeoffs', ',', 'landings'

[2021-06-09 17:00:37,992 INFO] 
SENT 2731: ['In', '2011', ',', 'company', 'officials', 'loaded', 'an', 'airliner', 'full', 'of', 'their', 'Kindle', 'e-readers', 'and', 'flew', 'it', 'around', 'to', 'test', 'for', 'problems', 'but', 'found', 'none', '.']
PRED 2731: In der Nähe der Europäischen Union in der Europäischen Union in der Europäischen Union in der Europäischen Union in der Europäischen Union .
PRED SCORE: -35.0958

[2021-06-09 17:00:37,992 INFO] 
SENT 2732: ['FAA', 'advisory', 'committee', 'members', 'expressed', 'mixed', 'feelings', 'about', 'whether', 'use', 'of', 'the', 'devices', 'presents', 'any', 'risk', '.']
PRED 2732: In der Nähe der Kommission in der Nähe der Kommission in der Nähe der Kommission .
PRED SCORE: -32.4316

[2021-06-09 17:00:37,993 INFO] 
SENT 2733: ['Douglas', 'Kidd', 'of', 'the', 'National', 'Association', 'of', 'Airline', 'Passengers', 'said', 'he', 'believes', 'interference', 'from', 'the', 'devices', 'is', 'genuine', 'even', 'if', 'the', 'risk', 'is'

In [12]:
# Detokenization : BPE tokenizing을 해준이후 쓸모없는 token들을 제거 해줌 
!sed -i "s/@@ //g"  OpenNMT-py/data/pred_1000.txt

In [11]:
# OpenNMT의 tools에 bleu 측정 tool을 사용 
!perl  OpenNMT-py/tools/multi-bleu.perl OpenNMT-py/data/tgt-val.txt < OpenNMT-py/data/pred_1000.txt

BLEU = 0.15, 11.7/0.5/0.0/0.0 (BP=0.718, ratio=0.751, hyp_len=49562, ref_len=65992)
